In [1]:
# # Run it once (in one session)
# !pip install decord
# !pip install einops
# !pip install icecream
# !pip install rarfile
# !pip install unrar

In [2]:
# Imports
import torch
from torch import nn, einsum
from torch.nn import functional as F
from torch.utils.tensorboard import SummaryWriter
import torchvision as tv
from torch.utils.data import random_split, DataLoader,Dataset
import time
import random
import math
import decord
import numpy as np
import gc
from einops import rearrange, repeat,reduce
from einops.layers.torch import Rearrange
from PIL import Image
from tqdm.notebook import tqdm
from icecream import ic
from torchvision.datasets import DatasetFolder
from torchvision.transforms import transforms
from torch.utils.data import DataLoader, random_split
import os
import rarfile
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from torch.utils.data import DataLoader, random_split
import torch.utils.data as data
from torchvision import transforms
from PIL import Image
import torch.nn as nn
import torch.optim as optim
from tqdm.notebook import tqdm
import shutil

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# set device
device ='cuda:0' if torch.cuda.is_available() else 'cpu'
device

'cuda:0'

# DOWNLOADING DATA FROM SOURCE WEBSITE

In [4]:
# # once per session/runtime
# !wget http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar 

# DATA DIRECTORY SETTINGS

In [5]:
# # once per session/runtime

# rar_path = './hmdb51_org.rar'  
# extract_path = './dataset' 

# with rarfile.RarFile(rar_path, 'r') as rar:
#     rar.extractall(extract_path)

In [6]:
# # once per session/runtime

# direcs = os.listdir(extract_path)

# for i in direcs:
#   with rarfile.RarFile(f"dataset/{i}", 'r') as rar:
#     rar.extractall(f"data/{i.split('.')[0]}")

In [7]:
# once per session/runtime



In [8]:
# once per session/runtime

# Define the path to the dataset


In [9]:
# data_path = './data'  # Path to the original dataset
# dataset_dir = './data_set'  # Destination directory for the modified dataset

# # Iterate over the class folders in the original dataset
# for class_folder in os.listdir(data_path):
#     class_folder_path = os.path.join(data_path, class_folder)

#     # Iterate over the subfolders within each class folder
#     for subfolder in os.listdir(class_folder_path):
#         subfolder_path = os.path.join(class_folder_path, subfolder)

#         # Iterate over the files within each subfolder
#         for file_name in os.listdir(subfolder_path):
#             file_path = os.path.join(subfolder_path, file_name)

#             # Create the destination directory in the modified dataset
#             dest_dir = os.path.join(dataset_dir, subfolder)
#             os.makedirs(dest_dir, exist_ok=True)

#             # Move the file to the destination directory
#             shutil.move(file_path, dest_dir)

# print("Dataset modification complete.") # success message

# DATA LOADING

In [10]:
# Dataset Class
class HMDB51Dataset(data.Dataset):
    def __init__(self, dataset_dir, frames_per_clip):
        super().__init__()
        self.dataset_dir = dataset_dir
        self.frames_per_clip = frames_per_clip
        self.video_list = []
        self.labels = []

        # Get the list of video directories
        video_dirs = sorted(os.listdir(dataset_dir))
        
        for label, video_dir in enumerate(video_dirs):
            video_files = os.listdir(os.path.join(dataset_dir, video_dir))
            self.video_list.extend([os.path.join(video_dir, video_file) for video_file in video_files])
            self.labels.extend([label] * len(video_files))
        #print(self.labels)
        self.transform = transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

    def __len__(self):
        return len(self.video_list)

    def __getitem__(self, idx):
        video_path = os.path.join(self.dataset_dir, self.video_list[idx])
        vid = decord.VideoReader(video_path, ctx=decord.cpu(0))
        nframes = len(vid)

        # If the number of frames in the video is less than frames_per_clip, repeat the frames
        if nframes <= self.frames_per_clip:
            print('not enough frame')
            frame_idxs = torch.arange(0, self.frames_per_clip) % nframes
        # Else, sample uniformly separated frames
        else:
            frame_idxs = torch.linspace(0, nframes - 1, self.frames_per_clip).long()        #do something here??

        frames = []
        for frame_idx in frame_idxs:
            frame_idx = frame_idx.item()  # Convert to scalar value
            frame = Image.fromarray(vid[frame_idx].asnumpy())
            frame = self.transform(frame)
            frames.append(frame)

        frames = torch.stack(frames)
        #print('frams shape is ',frames.shape)
        #print('frams is ',self.frames_per_clip)
        label = self.labels[idx]
        return frames, label

In [11]:
# Directory of the HMDB51 dataset
dataset_dir = "./data"
# Data loading parameters
batch_size = 32
test_batch_size = 1
num_workers = 0
pin_memory = True
frames_per_clip = 8
# Instantiate the dataset
hmdb51_dataset = HMDB51Dataset(dataset_dir,frames_per_clip=frames_per_clip)
num_classes = len(set(hmdb51_dataset.labels))

# TRAIN TEST SPLIT

In [12]:
# Split the dataset into train, validation, and test sets
train_len = int(0.7 * len(hmdb51_dataset))
test_len = len(hmdb51_dataset) - train_len - (len(hmdb51_dataset) - train_len)//2
train_data, val_data,test_data = torch.utils.data.random_split(hmdb51_dataset, [train_len,test_len, len(hmdb51_dataset) - test_len - train_len])





In [13]:
# Dataloaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=pin_memory,drop_last=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, num_workers=num_workers, pin_memory=pin_memory,drop_last=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=test_batch_size, num_workers=num_workers, pin_memory=pin_memory,drop_last=True)

In [14]:
# Instantiate and create train-val-test split
# train_val_data = HMDB51Dataset(dataset_dir, frames_per_clip=frames_per_clip)
# train_len = int(0.7 * len(hmdb51_dataset))
# test_len = len(hmdb51_dataset) - train_len - (len(hmdb51_dataset) - train_len)//2
# train_val_split = [train_len,test_len, len(hmdb51_dataset) - test_len - train_len]
# train_data, val_data,test_data = random_split(train_val_data, train_val_split)
# test_data = HMDB51Dataset(dataset_dir, frames_per_clip=frames_per_clip)

# Print the number of samples in each split
print(f"Train samples: {len(train_data)}")
print(f"Validation samples: {len(val_data)}")
print(f"Test samples: {len(test_data)}")

Train samples: 4736
Validation samples: 1015
Test samples: 1015


# DEFINING MODEL

In [15]:
class MLP(nn.Module):
    """
    Builds a simple feed forward network
    Args:
    - dim: (int) - inner dimension of embeddings
    - inner_dim: (int) - dimension of transformer head
    - n_class: (int) - number of output classes
    - encoder: the DinoVisionTransformer encoder
    """
    def __init__(self, dim, inner_dim, n_class, encoder):
        super().__init__()
        self.encoder = encoder
        self.mlp = nn.Sequential(
            nn.Flatten(),
            nn.Linear(dim*frames_per_clip,2048),
            nn.Linear(2048,512),
            nn.Linear(512, n_class)
        )

    def forward(self, x,concat=False):
        B, T, C, H, W = x.shape
        x = x.reshape(B*T, C, H, W)
        output = self.encoder(x)
        if concat:
            output = output.reshape(B, T, -1)
            avg = output.mean(dim=1)  # Average pooling over time
        else:
            avg = output.reshape(B, T, -1)
            #print(avg.shape)
        return self.mlp(avg)

In [16]:
# Instantiate the model
dinov2_vits14 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14')
dinov2_vits14.to(device)
for param in dinov2_vits14.parameters():
    param.requires_grad = False
# for i in range(12):
#     #dinov2_vits14.blocks[i].add_module('ln',nn.LayerNorm(384))
#     dinov2_vits14.blocks[i].add_module('fc1',nn.Linear(384, 384, bias=True))
#     dinov2_vits14.blocks[i].add_module('conv',nn.Conv3d(384, 384, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0), groups=384))
#     dinov2_vits14.blocks[i].add_module('fc2',nn.Linear(384, 384, bias=True))
model = MLP(384, 512, 51, dinov2_vits14)
model.to(device)

Using cache found in /home/z3qian/.cache/torch/hub/facebookresearch_dinov2_main


MLP(
  (encoder): DinoVisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 384, kernel_size=(14, 14), stride=(14, 14))
      (norm): Identity()
    )
    (blocks): ModuleList(
      (0-11): 12 x NestedTensorBlock(
        (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
        (attn): MemEffAttention(
          (qkv): Linear(in_features=384, out_features=1152, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=384, out_features=384, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (ls1): LayerScale()
        (drop_path1): Identity()
        (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=384, out_features=1536, bias=True)
          (act): GELU(approximate='none')
          (fc2): Linear(in_features=1536, out_features=384, bias=True)
          (drop): Dropout(p=0.0, inplace=False)
        )
        (

## SPECIFYING HYPER-PARAMETERS

In [17]:
# Define the loss function and optimizer
lr=0.01
epochs = 10
loss_criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=lr,momentum=0.9,weight_decay=0.01)

# MODEL TRAINING & EVALUATION

In [18]:
# Training loop
val_best = 0
for epoch in range(1, epochs + 1):
    model.train()
    total_epoch_loss = 0
    for batch_id, (video_data, labels) in tqdm(enumerate(train_loader)):
        video_data, labels = video_data.to(device), labels.to(device)
        optimizer.zero_grad()
        prediction = model(video_data,False)
        loss = loss_criterion(prediction, labels)
        loss.backward()
        optimizer.step()
        total_epoch_loss += loss.item()
        
        corrects = (torch.argmax(prediction,dim=1)==labels).sum()
        bacc = corrects/batch_size
        del video_data
        del labels

        gc.collect()
        print(f"\n[Train Epoch]: {epoch} Train Loss: {loss.item()}, Batch Acc is {bacc.item()}")

        # Add any additional training metrics/logging you need
        
    # Perform validation at the end of each epoch
    model.eval()
    total_loss = 0
    corrects = 0
    with torch.no_grad():
        for batch_id, (video_data, labels) in enumerate(val_loader):
            video_data, labels = video_data.to(device), labels.to(device)
            prediction = model(video_data)
            loss = loss_criterion(prediction, labels)
            total_loss += loss.item()
            corrects += (torch.argmax(prediction, dim=1) == labels).sum()
            del video_data
            del labels

            gc.collect()
    accuracy = corrects / (len(val_loader) * batch_size)
    print(f"\n[Val Epoch]: {epoch} , Accuracy: {accuracy}, Valid Loss: {total_loss / len(val_loader)}")
    if accuracy > val_best:
        torch.save(model,'best_hmdb_model.pth')
        val_best = accuracy

0it [00:00, ?it/s]


[Train Epoch]: 1 Train Loss: 4.033176422119141, Batch Acc is 0.0

[Train Epoch]: 1 Train Loss: 4.068737506866455, Batch Acc is 0.0625

[Train Epoch]: 1 Train Loss: 3.775747537612915, Batch Acc is 0.125

[Train Epoch]: 1 Train Loss: 3.4547877311706543, Batch Acc is 0.28125

[Train Epoch]: 1 Train Loss: 3.3328020572662354, Batch Acc is 0.25

[Train Epoch]: 1 Train Loss: 3.4137349128723145, Batch Acc is 0.3125

[Train Epoch]: 1 Train Loss: 4.62029504776001, Batch Acc is 0.125

[Train Epoch]: 1 Train Loss: 3.0617599487304688, Batch Acc is 0.28125

[Train Epoch]: 1 Train Loss: 4.720509052276611, Batch Acc is 0.3125

[Train Epoch]: 1 Train Loss: 3.8631398677825928, Batch Acc is 0.3125

[Train Epoch]: 1 Train Loss: 3.3075873851776123, Batch Acc is 0.4375

[Train Epoch]: 1 Train Loss: 3.042165517807007, Batch Acc is 0.34375

[Train Epoch]: 1 Train Loss: 3.014737129211426, Batch Acc is 0.40625

[Train Epoch]: 1 Train Loss: 2.3650078773498535, Batch Acc is 0.46875

[Train Epoch]: 1 Train Loss: 

0it [00:00, ?it/s]


[Train Epoch]: 2 Train Loss: 9.247452735900879, Batch Acc is 0.6875

[Train Epoch]: 2 Train Loss: 16.227087020874023, Batch Acc is 0.4375

[Train Epoch]: 2 Train Loss: 11.892719268798828, Batch Acc is 0.46875

[Train Epoch]: 2 Train Loss: 5.773836135864258, Batch Acc is 0.59375

[Train Epoch]: 2 Train Loss: 11.80241584777832, Batch Acc is 0.5

[Train Epoch]: 2 Train Loss: 13.989668846130371, Batch Acc is 0.3125

[Train Epoch]: 2 Train Loss: 6.756494998931885, Batch Acc is 0.6875

[Train Epoch]: 2 Train Loss: 12.47114372253418, Batch Acc is 0.46875

[Train Epoch]: 2 Train Loss: 22.0532283782959, Batch Acc is 0.4375

[Train Epoch]: 2 Train Loss: 14.830195426940918, Batch Acc is 0.5625

[Train Epoch]: 2 Train Loss: 15.486989974975586, Batch Acc is 0.34375

[Train Epoch]: 2 Train Loss: 11.630880355834961, Batch Acc is 0.4375

[Train Epoch]: 2 Train Loss: 14.438766479492188, Batch Acc is 0.4375

[Train Epoch]: 2 Train Loss: 12.731292724609375, Batch Acc is 0.5625

[Train Epoch]: 2 Train Lo

0it [00:00, ?it/s]

...


[Train Epoch]: 8 Train Loss: 122447.390625, Batch Acc is 0.65625

[Train Epoch]: 8 Train Loss: 171929.953125, Batch Acc is 0.46875

[Train Epoch]: 8 Train Loss: 240350.515625, Batch Acc is 0.4375

[Train Epoch]: 8 Train Loss: 188490.0625, Batch Acc is 0.4375

[Train Epoch]: 8 Train Loss: 205816.984375, Batch Acc is 0.40625

[Train Epoch]: 8 Train Loss: 140294.46875, Batch Acc is 0.46875

[Train Epoch]: 8 Train Loss: 207298.375, Batch Acc is 0.40625

[Train Epoch]: 8 Train Loss: 154489.8125, Batch Acc is 0.625

[Train Epoch]: 8 Train Loss: 152691.984375, Batch Acc is 0.46875

[Train Epoch]: 8 Train Loss: 198491.328125, Batch Acc is 0.34375

[Train Epoch]: 8 Train Loss: 171406.140625, Batch Acc is 0.5

[Train Epoch]: 8 Train Loss: 207686.796875, Batch Acc is 0.53125

[Train Epoch]: 8 Train Loss: 113047.5234375, Batch Acc is 0.5625

[Train Epoch]: 8 Train Loss: 137575.46875, Batch Acc is 0.59375

[Train Epoch]: 8 Train Loss: 196591.921875, Batch Acc is 0.46875

[Train Epoch]: 8 Train Los

0it [00:00, ?it/s]


[Train Epoch]: 9 Train Loss: 1011315.6875, Batch Acc is 0.40625

[Train Epoch]: 9 Train Loss: 705956.8125, Batch Acc is 0.46875

[Train Epoch]: 9 Train Loss: 837022.1875, Batch Acc is 0.46875

[Train Epoch]: 9 Train Loss: 515928.6875, Batch Acc is 0.5625

[Train Epoch]: 9 Train Loss: 847206.6875, Batch Acc is 0.375

[Train Epoch]: 9 Train Loss: 990247.9375, Batch Acc is 0.4375

[Train Epoch]: 9 Train Loss: 879286.625, Batch Acc is 0.4375

[Train Epoch]: 9 Train Loss: 886751.0, Batch Acc is 0.5

[Train Epoch]: 9 Train Loss: 1193518.625, Batch Acc is 0.40625

[Train Epoch]: 9 Train Loss: 999584.8125, Batch Acc is 0.5

[Train Epoch]: 9 Train Loss: 815231.0, Batch Acc is 0.5625

[Train Epoch]: 9 Train Loss: 1188788.25, Batch Acc is 0.375

[Train Epoch]: 9 Train Loss: 580877.5, Batch Acc is 0.53125

[Train Epoch]: 9 Train Loss: 1001141.625, Batch Acc is 0.375

[Train Epoch]: 9 Train Loss: 1451644.375, Batch Acc is 0.4375

[Train Epoch]: 9 Train Loss: 1339342.875, Batch Acc is 0.28125

[Tra

0it [00:00, ?it/s]


[Train Epoch]: 10 Train Loss: 3472439.75, Batch Acc is 0.5

[Train Epoch]: 10 Train Loss: 3824686.25, Batch Acc is 0.5625

[Train Epoch]: 10 Train Loss: 6377182.5, Batch Acc is 0.46875

[Train Epoch]: 10 Train Loss: 8904964.0, Batch Acc is 0.3125

[Train Epoch]: 10 Train Loss: 4487330.0, Batch Acc is 0.53125

[Train Epoch]: 10 Train Loss: 4738178.5, Batch Acc is 0.40625

[Train Epoch]: 10 Train Loss: 4243810.5, Batch Acc is 0.59375

[Train Epoch]: 10 Train Loss: 4543042.0, Batch Acc is 0.40625

[Train Epoch]: 10 Train Loss: 4254726.5, Batch Acc is 0.40625

[Train Epoch]: 10 Train Loss: 5197490.5, Batch Acc is 0.3125

[Train Epoch]: 10 Train Loss: 4256863.0, Batch Acc is 0.46875

[Train Epoch]: 10 Train Loss: 4008669.75, Batch Acc is 0.53125

[Train Epoch]: 10 Train Loss: 6687360.0, Batch Acc is 0.375

[Train Epoch]: 10 Train Loss: 6544338.0, Batch Acc is 0.5625

[Train Epoch]: 10 Train Loss: 9179180.0, Batch Acc is 0.3125

[Train Epoch]: 10 Train Loss: 3864655.75, Batch Acc is 0.5625


In [19]:
# Save the trained model
#torch.save(model, "hmdb_st_model.pth")
model = torch.load('best_hmdb_model.pth')
model.to(device)

MLP(
  (encoder): DinoVisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 384, kernel_size=(14, 14), stride=(14, 14))
      (norm): Identity()
    )
    (blocks): ModuleList(
      (0-11): 12 x NestedTensorBlock(
        (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
        (attn): MemEffAttention(
          (qkv): Linear(in_features=384, out_features=1152, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=384, out_features=384, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (ls1): LayerScale()
        (drop_path1): Identity()
        (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=384, out_features=1536, bias=True)
          (act): GELU(approximate='none')
          (fc2): Linear(in_features=1536, out_features=384, bias=True)
          (drop): Dropout(p=0.0, inplace=False)
        )
        (

In [20]:
def test_model(loader):

    model.eval()
    corrects=0
    total_loss = 0
    with torch.no_grad():
        for batch_id, (input_data,labels) in enumerate(loader):
            
            input_data = input_data.to(device)
            
            labels = labels.to(device)
            prediction = model(input_data)
            loss = loss_criterion(prediction,labels)
            total_loss += loss.item()
            corrects+= (torch.argmax(prediction,dim=1)==labels).sum()
    
    accuracy = corrects/(len(loader)*test_batch_size)
    print(f"Test Accuracy: {accuracy}, Test Loss: {total_loss}")

    return accuracy

In [21]:
test_model(test_loader)

Test Accuracy: 0.40788179636001587, Test Loss: 71269.69884929057


tensor(0.4079, device='cuda:0')